In [5]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

In [2]:
df = pd.read_pickle('movies_df.zip')

In [ ]:
df.to_csv('movies.csv', index=False)

In [3]:
train, test = train_test_split(df, test_size=0.15, random_state = 5)

In [6]:
Clean = False
Train = True

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en")

In [8]:
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0001)

In [9]:
def model_train():
    model.train()
    losses = 0
    X = train['fa']
    y = train['ro']
    max_epochs = 2
    n_batches = 3
    for epoch in tqdm(range(max_epochs)):
        for i in tqdm(range(125)):
            # making batches 
            local_X, local_y = X[i*n_batches:(i+1)*n_batches,], y[i*n_batches:(i+1)*n_batches,]
            # preparing the data according to the model input
            batch = tokenizer.prepare_seq2seq_batch(list(local_X),list(local_y),return_tensors='pt')
            output = model(**batch)
            # loss can be taken directly from the model output
            loss = output.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses = losses+loss
    average = losses/len(df)
    print('Loss: ' + str(average) )
    
    return model

In [10]:
model = model_train()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

E:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


  0%|          | 0/125 [00:00<?, ?it/s]

Loss: tensor(0.2648, grad_fn=<DivBackward0>)


In [95]:
epoch = 3
torch.save(model.state_dict(), f'{epoch}.mt')

In [59]:
df_pred = pd.DataFrame(columns=['pred'])

In [60]:
for n, _ in enumerate(test.iloc[:,0]):
    a = model.generate(**tokenizer.prepare_seq2seq_batch(test.iloc[n,1],return_tensors='pt'))
    text = tokenizer.batch_decode(a)
    df_pred.loc[len(df_pred.index)] = pd.Series({'pred':text})

In [159]:
pred = []
for element in df_pred.iloc[:,0]:
    #pred.append(element[0])
    pred.append(word_tokenize(str(element)))

In [160]:
gt = []
for element in test.iloc[:,0]:
    #gt.append(element)
    gt.append([word_tokenize(str(element))])

In [149]:
test.iloc[:,0].to_csv('pred_fm3e.csv', index=False)
df_pred.iloc[:,0].to_csv('gt_fm3e.csv', index=False)

In [161]:
from torchtext.data.metrics import bleu_score

bleu_score(pred, gt)

0.00786789320409298